<a href="https://colab.research.google.com/github.com/HectorLob/wandb_edu/blob/main/mlops-001/lesson1/03_Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{course-lesson1} -->

# Baseline solution

<!--- @wandbcode{course-lesson1} -->

In this notebook we will create a baseline solution to our semantic segmentation problem. To iterate fast a notebook is a handy solution. We will then refactor this code into a script to be able to use hyperparameter sweeps.

In [4]:
import wandb
import pandas as pd
from fastai.vision.all import *
from fastai.callback.wandb import WandbCallback

import params
from utils import get_predictions, create_iou_table, MIOU, BackgroundIOU, \
                  RoadIOU, TrafficLightIOU, TrafficSignIOU, PersonIOU, VehicleIOU, BicycleIOU

Again, we're importing some global configuration parameters from `params.py` file. We have also defined some helper functions in `utils.py` - for example metrics we will track during our experiments.

Let's now create a `train_config` that we'll pass to W&B `run` to control training hyperparameters. 

In [5]:
train_config = SimpleNamespace(
    framework="fastai",
    img_size=(180, 320),
    batch_size=8,
    augment=True, # use data augmentation
    epochs=10, 
    lr=2e-3,
    pretrained=True,  # whether to use pretrained encoder
    seed=42,
)

We are setting seed for reproducibility. 

In [6]:
set_seed(train_config.seed, reproducible=True)

In [7]:
run = wandb.init(project=params.WANDB_PROJECT, entity=params.ENTITY, job_type="training", config=train_config)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: hlo (gaiusgaius). Use `wandb login --relogin` to force relogin


As usual, we will use W&B Artifacts to track the lineage of our models. 

In [8]:
processed_data_at = run.use_artifact(f'{params.PROCESSED_DATA_AT}:latest')
processed_dataset_dir = Path(processed_data_at.download())
df = pd.read_csv(processed_dataset_dir / 'data_split.csv')

wandb: Downloading large artifact bdd_simple_1k_split:latest, 813.24MB. 4010 files... 
wandb:   4010 of 4010 files downloaded.  
Done. 0:0:7.1


We will not use the hold out dataset stage at this moment. `is_valid` column will tell our trainer how we want to split data between training and validation. 

In [9]:
df = df[df.Stage != 'test'].reset_index(drop=True)
df['is_valid'] = df.Stage == 'valid'

In [10]:
def label_func(fname):
    return (fname.parent.parent/"labels")/f"{fname.stem}_mask.png"

We will use `fastai`'s `DataBlock` API to feed data into model training and validation. 

In [25]:
# assign paths
df["image_fname"] = [processed_dataset_dir/f'images/{f}.jpg' for f in df.File_Name.values]
df["label_fname"] = [label_func(f) for f in df.image_fname.values]

In [26]:
df

,File_Name,Stage,is_valid,image_fname,label_fname
0,3b76e313-a4f861d4,train,False,artifacts/bdd_simple_1k_split:v4/images/3b76e313-a4f861d4.jpg,artifacts/bdd_simple_1k_split:v4/labels/3b76e313-a4f861d4_mask.png
1,c665137e-6fffaf45,train,False,artifacts/bdd_simple_1k_split:v4/images/c665137e-6fffaf45.jpg,artifacts/bdd_simple_1k_split:v4/labels/c665137e-6fffaf45_mask.png
2,42041574-fb905492,train,False,artifacts/bdd_simple_1k_split:v4/images/42041574-fb905492.jpg,artifacts/bdd_simple_1k_split:v4/labels/42041574-fb905492_mask.png
3,587e93da-9d2eef22,train,False,artifacts/bdd_simple_1k_split:v4/images/587e93da-9d2eef22.jpg,artifacts/bdd_simple_1k_split:v4/labels/587e93da-9d2eef22_mask.png
4,a188e945-124b917e,train,False,artifacts/bdd_simple_1k_split:v4/images/a188e945-124b917e.jpg,artifacts/bdd_simple_1k_split:v4/labels/a188e945-124b917e_mask.png
...,...,...,...,...,...
895,1cbc652a-535d7ba3,train,False,artifacts/bdd_simple_1k_split:v4/images/1cbc652a-535d7ba3.jpg,artifacts/bdd_simple_1k_split:v4/labels/1cbc652a-535d7ba3_mask.png
896,53af996f-00000000,valid,True,artifacts/bdd_simple_1k_split:v4/images/53af996f-00000000.jpg,artifacts/bdd_simple_1k_split:v4/labels/53af996f-00000000_mask.png
897,6e9110c1-9973f5ab,valid,True,artifacts/bdd_simple_1k_split:v4/images/6e9110c1-9973f5ab.jpg,artifacts/bdd_simple_1k_split:v4/labels/6e9110c1-9973f5ab_mask.png
898,43d4a202-ca890610,train,False,artifacts/bdd_simple_1k_split:v4/images/43d4a202-ca890610.jpg,artifacts/bdd_simple_1k_split:v4/labels/43d4a202-ca890610_mask.png


In [27]:
def get_data(df, bs=4, img_size=(180, 320), augment=True):
    block = DataBlock(blocks=(ImageBlock, MaskBlock(codes=params.BDD_CLASSES)),
                  get_x=ColReader("image_fname"),
                  get_y=ColReader("label_fname"),
                  splitter=ColSplitter(),
                  item_tfms=Resize(img_size),
                  batch_tfms=aug_transforms() if augment else None,
                 )
    return block.dataloaders(df, bs=bs)

We are using `wandb.config` to track our training hyperparameters. 

In [28]:
config = wandb.config

In [29]:
dls = get_data(df, bs=config.batch_size, img_size=config.img_size, augment=config.augment)

We will use *intersection over union* metrics: mean across all classes (MIOU) and IOU for each class separately. Our model will be a `unet` based on pretrained `resnet18` backbone. 

In [30]:
metrics = [MIOU(), BackgroundIOU(), RoadIOU(), TrafficLightIOU(), \
           TrafficSignIOU(), PersonIOU(), VehicleIOU(), BicycleIOU()]

learn = unet_learner(dls, arch=resnet18, pretrained=config.pretrained, metrics=metrics)

/home/hector/miniconda3/envs/mlops_1/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hector/miniconda3/envs/mlops_1/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/hector/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:40<00:00, 1.16MB/s]


In `fastai` we already have a callback that integrates tightly with W&B, we only need to pass the `WandbCallback` to the learner and we are ready to go. The callback will log all the useful variables for us. For example, whatever metric we pass to the learner will be tracked by the callback.

In [31]:
callbacks = [
    SaveModelCallback(monitor='miou'),
    WandbCallback(log_preds=False, log_model=True)
]

Let's train our model!

In [32]:
learn.fit_one_cycle(config.epochs, config.lr, cbs=callbacks)

epoch,train_loss,valid_loss,miou,background_iou,road_iou,traffic_light_iou,traffic_sign_iou,person_iou,vehicle_iou,bicycle_iou,time
0,0.487695,0.333676,0.314746,0.866439,0.755305,0.000000,0.000000,0.000000,0.581476,0.000000,29:09
1,0.417438,0.332912,0.318155,0.869325,0.773027,0.000000,0.000000,0.000000,0.584734,0.000000,28:38
2,0.346751,0.351197,0.326309,0.887690,0.768932,0.000000,0.000000,0.000000,0.627539,0.000000,27:02


KeyboardInterrupt: 

We will log a table with model predictions and ground truth to W&B, so that we can do error analysis in the W&B dashboard. 

In [33]:
samples, outputs, predictions = get_predictions(learn)
table = create_iou_table(samples, outputs, predictions, params.BDD_CLASSES)
wandb.log({"pred_table":table})

We are reloading the model from the best checkpoint at the end and saving it. To make sure we track the final metrics correctly, we will validate the model again and save the final loss and metrics to `wandb.summary`. 

In [34]:
scores = learn.validate()
metric_names = ['final_loss'] + [f'final_{x.name}' for x in metrics]
final_results = {metric_names[i] : scores[i] for i in range(len(scores))}
for k,v in final_results.items(): 
    wandb.summary[k] = v

In [35]:
wandb.finish()

background_iou,▁▂█
bicycle_iou,▁▁▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
eps_0,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eps_1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eps_2,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr_0,▁▁▁▁▁▁▂▂▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇▇█████████████
lr_1,▁▁▁▁▁▁▂▂▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇▇█████████████
lr_2,▁▁▁▁▁▁▂▂▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇▇█████████████
miou,▁▃█
mom_0,██████▇▇▇▇▇▆▆▆▅▅▅▄▄▄▃▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
